In [ ]:
 import torch
import torch.nn as nn
import torch.nn.functional as F 
 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm 
import matplotlib.pyplot as plt
import numpy as np

In this homework, we will explore deep networks using  PyTorch framework!
With the MNIST dataset consisting of 70000 images (28 * 28 * 1) of handwritten digits, we will train a multiple-layer perceptron network to detect the digit corresponding to the input image. 

#Data Loading

Run the cell below to download the dataset. 

In [ ]:

mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='./data', train=False, download=True,transform=transforms.ToTensor()) 


In [ ]:
mnist_train

In [ ]:
mnist_test

In [ ]:
image, label = mnist_test[0]
label

In [ ]:
image.shape

In [ ]:
plt.imshow(image.reshape(28,28), cmap='gray')

#Model


The model we want to classify the MNIST dataset is a two-hidden-layer MLP with 120 and 84 units in each layer, respectively. 

Create a PyTorch model for described model and use ReLU for the activation function.




In [ ]:
class MultilayerPerceptron(nn.Module):

    #todo


In [ ]:
torch.manual_seed(80)
model = MultilayerPerceptron()
model

Answer these questions:
2. Denote the size of the weights matrix of each layer (consider the input and output layer as well). 
1. What initial values are used  ​​for the weights and biases of your model? (Do not report the specific value, we are interested in distributions that are used.)

#Training


In this section, we want to train our MLP model using the SGD algorithm. 

As we discussed in class, instead of calculating loss using the whole inputs, we break the dataset (with size $n$) into some batches of size m ($m < n$) and, at each iteration, calculate the loss of one batch and update the weights.  

Using the `DataLoader` class of the torch library, complete the cell below to batch the test and train dataset.

In [ ]:
batch_size = 100
train_loader = #todo
test_loader = #todo

there are yet a couple of hyperprameters that we should define before training.

In [ ]:

epochs = 10
learning_rate = 0.01 #feel free to play with this


Answer these questions:
* Prove that the **expected value** of the gradient obtained using a random batch equals the gradient of the whole dataset.

* For the dataset of size $n = $12000, if the batch size is 200, what would be the number of weight updates (or iterations) in a total of 5 epoche?


Complete the training function below, such that at each epoch:




1. For each batch:

  1. Perform feedforward
  2. Calculate loss (use cross-entropy loss)
  3. Perform backpropagation and updating weights

2. Calculate and report the training accuracy and loss for each epoch (use the average of the batch accuracy as epoch accuracy).
3. Calculate and report the test accuracy and loss for each epoch


In [ ]:
#initialize loss function and optimizer
loss_function = #todo
optimizer = #todo

Highly encourage you to use [this](https://github.com/tqdm/tqdm) awesome package called taqaddum to have a nice progress bar for the training process


In [ ]:

def train(model, loss_function, optimizer, train_loader, test_loader, epochs):

    train_losses = []
    test_losses = []
    train_acc = []
    test_acc  = []



    #todo



    return train_losses, train_acc, test_acc, test_losses

In [ ]:
train_losses, train_acc, test_acc, test_losses = train(model, loss_function, optimizer, train_loader, test_loader, epochs)

Now plot the training results (one plot for test and training accuracy and one for the loss)

In [ ]:
#plot loss


In [ ]:
#plot accuracy


Based on your plots, answer these questions:
1. Compare the training and test loss
2. Did your model become overfitted? Why?
3. Why your training loss is not a  decreasing function with respect to #epoch?

Repeat the training process for 5 diffrent learning rate and compare the results.

In [ ]:
#to do

#Vanishing Gradient

In this section, we want to see the vanishing gradient problem.

Create two separate models with 5 hidden layers of size [100, 100, 80, 50, 30],
one of them uses ReLU as an activation function and the other one uses Sigmoid.


In [ ]:
class MLP_relu(nn.Module):

  #todo

In [ ]:
class MLP_sigmoid(nn.Module):

  #todo


In [ ]:
#instantiate the models


Modify the train function to save the gradient and weights of each layer, every epoch. Return these values as well. 

In [ ]:
def train(model, loss_function, optimizer, train_loader, test_loader, epochs):

    train_losses = []
    test_losses = []
    train_acc = []
    test_acc  = []
    weights = {'H1':[], 'H2':[], 'H3':[],'H4':[], 'H5':[]}
    biases = {'H1':[], 'H2':[], 'H3':[],'H4':[], 'H5':[]}
    grads = {'H1':[], 'H2':[], 'H3':[],'H4':[], 'H5':[]}


    #todo



    return train_losses, train_acc, test_acc, test_losses, weights, grads

Now, train the sigmoid and relu model.

In [ ]:
#training

In [ ]:
#plot loss


In [ ]:
#plot accuracy

As you can see, the sigmoid model is not good at all. Based on the following plots explain why.

For each layer, calculate the mean and std (standard deviation) of all its weights and plot these values ​​vs #epochs. Repeat the same for the grads.

You should do:

* Calculate the average weight of each hidden layer in each epoch.
* Calculate the std for the weight of each hidden layer in each epoch.

* Calculate the average grads of each hidden layer in each epoch.
* Calculate the std for the grads of each hidden layer in each epoch.

Plot all of them for both models (use subplot to compare two models' results).






In [ ]:
#calcute means and stds

In [ ]:
#plot weights mean for the both models

In [ ]:
#plot weights std for the both models

In [ ]:
#plot grads mean for the both models

In [ ]:
#plot grads std for the both models

Good Luck!